### **Content License Agreement**

<font color='red'><b>**WARNING**</b></font> : 본 자료는 삼성 청년 SW아카데미의 컨텐츠 자산으로, 보안서약서에 의거하여 어떠한 사유로도 임의로 복사, 촬영, 녹음, 복제, 보관, 전송하거나 허가 받지 않은 저장매체를 이용한 보관, 제3자에게 누설, 공개 또는 사용하는 등의 무단 사용 및 불법 배포 시 법적 조치를 받을 수 있습니다.

## **Objectives**
본 실습의 목표를 기입합니다. 아래 내용을 반드시 포함합니다.
1. 실습 개요
    - SERPAPI (실시간 검색 엔진)을 활용하여 최신 정보를 수집하는 기능 구현
    - 수집한 검색 결과를 DB에 저장
    - LangGraph를 사용하여 고도화된 RAG 파이프라인 구축

2. 실습 진행 목적 및 배경
    - 목적: 실시간 검색과 이를 LangGraph로 구현함으로써 RAG 파이프라인을 고도화한다.
    - 배경: 실시간으로 데이터를 받아서 활용하는 것 역시 중요해지고 있으며 성능 개선을 위해 RAG의 고도화된 워크플로우를 구축하는 것이 중요해지고 있습니다.

3. 실습 수행으로 얻어갈 수 있는 역량
    - SERPAPI 검색 엔진을 연동한 최신 정보 접근 및 활용 기술.
    - LangGraph를 활용한 RAG 워크플로우 설계 및 구현 능력.
    

4. 실습 핵심 내용
    - SERPAPI 활용: 검색 결과를 JSON 형식으로 수집.
    - LangGraph를 활용한 워크플로우: 복잡한 다중 경로 RAG 워크플로우 설계 및 최적화.

## **Prerequisites**


```
langchain == 0.3.12
langchain-chroma == 0.1.4
langchain-core == 0.3.25
langchain-openai == 0.2.12
langchain-text-splitters == 0.3.3
langchain-upstage == 0.4.0
getpass4 == 0.0.14.1
openai == 1.57.4
ragas == 0.2.8
langchain_community == 0.3.12
unstructured == 0.16.11
langgraph == 0.2.59
google-search-results == 2.4.2
```



In [ ]:
# 약 1분 소요
!pip install -qU openai langchain langchain-upstage langchain-chroma getpass4 langchain-community unstructured langgraph google-search-results

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 15.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.3/454.3 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# @title set Upstage API key
import os
import getpass
import warnings

warnings.filterwarnings("ignore")

# Get the Upstage API key using getpass
try:
    if "UPSTAGE_API_KEY" not in os.environ or not os.environ["UPSTAGE_API_KEY"]:
        os.environ["UPSTAGE_API_KEY"] = getpass.getpass("Enter your Upstage API key: ")

    print("API key has been set successfully.")

except:
    print("Something wrong with your API KEY. Check your API Console again.")

Enter your Upstage API key: ··········
API key has been set successfully.


#### SerpAPI API Key
- [SERPAPI](https://serpapi.com/) 회원 가입 후 API를 발급 받이 이용바랍니다.

In [ ]:
from serpapi import GoogleSearch

# SERPAPI KEY
if not os.getenv("SERPAPI_API_KEY"):
    GoogleSearch.SERP_API_KEY = getpass.getpass("Enter your SERPAPI API key: ")
    os.environ["SERPAPI_API_KEY"] = GoogleSearch.SERP_API_KEY

Enter your SERPAPI API key: ··········


<font color='red'><b>**Caution**</b></font>
- SERPAPI는 한달에 100개의 검색을 무료로 사용할 수 있습니다. 지나치게 많이 활용할 경우, 사용 제한이 걸릴 수 있습니다.

## **Exercise Overview**

실습 목차
- 실시간 검색 기능 RAG 시스템 구축 실습: 검색 엔진을 붙여 실시간 검색 기능을 활용하고, LangGraph을 사용하여 고도화된 형태의 RAG 파이프라인을 구축합니다.
    - 1) 실시간 검색 기능 활용
    - 2) LangGraph를 활용하여 고도화된 RAG 파이프라인 구축하기

## 1) 실시간 검색 기능 활용

- SERPAPI 검색 엔진을 활용하여 실시간 검색을 수행합니다.
- 검색 엔진을 결합하여 질의응답을 수행합니다.

#### 1-1) SERPAPI 검색 엔진을 활용하여 실시간 검색

In [ ]:
from serpapi import GoogleSearch

# 3강에서 활용한 논문(openai-o1-system-card)과 관련된 내용을 검색합니다.
# TODO: 다음 질문들에 대한 검색 결과를 확인합니다.
questions = [
"What distinguishes the o1 model’s reasoning capabilities from previous OpenAI models, and how does 'chain-of-thought' improve its performance?",
'How does the o1 model perform in jailbreak evaluations compared to GPT-4o, and what measures have been implemented to resist adversarial prompts?',
'What are the key improvements in the o1 model regarding hallucinations and fairness compared to GPT-4o, and how were these measured?',
'What safety challenges arise from o1’s enhanced reasoning abilities, and how does OpenAI address the risks associated with chain-of-thought reasoning?',
'What were the findings from external red teaming efforts, particularly concerning the o1 model’s susceptibility to manipulation, persuasion, and scheming behaviors?'
]

params = {
    "engine": "google",
    "q": questions[0],
    "num": "4"
}
search = GoogleSearch(params)

In [ ]:
search_result = search.get_dict()

In [ ]:
# 수집된 링크의 내용을 수집합니다! 파싱된 데이터를 검색시 활용되는 데이터로 활용할 수 있습니다.
from langchain.document_loaders import UnstructuredURLLoader

# URL은 여러개 사용할 수 있습니다.
urls = [result['link'] for result in search_result['organic_results']]

loader = UnstructuredURLLoader(urls=urls)
data = loader.load()

In [ ]:
data[0]

Document(metadata={'source': 'https://medium.com/@researchgraph/how-openais-o1-series-stands-out-redefining-ai-reasoning-9e499937139e'}, page_content='Open in app\n\nSign in\n\nWrite\n\nSign in\n\nHow OpenAI’s O1 Series Stands Out Redefining AI Reasoning\n\nExploring whether OpenAI’s O1 models represent a true breakthrough in AI reasoning or just incremental progress\n\nResearch Graph\n\nFollow\n\n19 min read\n\nSep 20, 2024\n\n--\n\nAuthor\n\nZijian Yang (ORCID: 0009–0006–8301–7634)\n\nIntroduction\n\nOpenAI’s recently released O1 series models signify a significant step forward in the field of artificial intelligence, particularly in complex reasoning. The O1 model series not only handles more intricate problems but also introduces human-like thought processes, elevating the model’s reasoning capabilities to new heights. In this article, we will delve into the core features of the O1 series models.\n\nThe O1 series models from OpenAI include two main versions, each designed to meet d

#### 1-2) 검색 엔진을 결합하여 질의응답을 수행합니다.

In [ ]:
from langchain_upstage import ChatUpstage
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

template = """Answer the question based on context.

Question: {question}
Context: {context}
Answer:"""

llm = ChatUpstage()
prompt = ChatPromptTemplate.from_template(template)
parser = StrOutputParser()

chain = (
    {"question": RunnablePassthrough(), "context": RunnablePassthrough()}
    | prompt
    | llm
    | parser
)

In [ ]:
# TODO: 다른 질문들에 대해서도 답변이 옳게 나왔는 지 확인해봅니다.
print(f"질문: {questions[0]}")
print("-" * 20)
print(f"답변:\n{chain.invoke({'question': questions[0], 'context': data[0].page_content})}")
print(f"답변:\n{chain.invoke({'question': questions[0], 'context': data[0].page_content}, encode='utf-8')}")

질문: What distinguishes the o1 model’s reasoning capabilities from previous OpenAI models, and how does 'chain-of-thought' improve its performance?
--------------------
답변:
The context does not provide specific information about what distinguishes the O1 model’s reasoning capabilities from previous OpenAI models or how the 'chain-of-thought' improves its performance.
답변:
The O1 model’s reasoning capabilities are distinguished from previous OpenAI models by its ability to handle more intricate problems and introduce human-like thought processes, elevating the model’s reasoning capabilities to new heights. The 'chain-of-thought' approach improves the model's performance by introducing a human-like thought process, allowing the model to reason through problems in a more comprehensive and systematic manner.


#### 1-3) LangChain으로 SERPAPI 연동해서 사용하기

In [ ]:
# LangChain을 활용할 경우, Snippet만을 가져와서 사용합니다. (Snippet: 간략한 미리보기 텍스트)
from langchain_community.utilities import SerpAPIWrapper

search = SerpAPIWrapper(params=params)
search.run(questions[0])

'[\'Compared to previous models, O1 features a more robust self-correction mechanism and adaptability.\', "One of the standout features of o1 is its ability to chain thoughts together, which means it\'s better equipped to tackle multi-step problems ...", \'A standout feature of the o1 series is its ability to reason through problems using a chain of thought. This approach mimics human problem- ...\', "Let\'s talk about how the model was trained, what makes it different from past LLMs, and why it will probably feel like a bit of a letdown for most users."]'

In [ ]:
# 앞서 파싱한 결과물과 비교합니다.
search_result['organic_results'][0]['snippet']

'Compared to previous models, O1 features a more robust self-correction mechanism and adaptability.'

## 2) LangGraph를 활용하여 고도화된 RAG 파이프라인 구축하기

- [LangGraph](https://langchain-ai.github.io/langgraph/) 페이지를 참고하여 학습합니다.
- 다양한 RAG 고도화 기술 중 CRAG(Corrective RAG)에 대하여 직접 구현해봅니다.

![image](https://github.com/user-attachments/assets/532a17d2-bc80-46e6-9b24-05de5fb3894b)


1. 문서를 다운받고 DB 및 검색기를 생성합니다.
2. **Retrieve**: 질문에 관련된 문서를 검색합니다.
3. **Grade**: 검색된 문서가 적절한 지 판단합니다.
4. **Generate**: 적절할 경우, 질문과 함께 LLM에 입력하여 답변을 생성합니다.
5. **Re-Write**: 적절하지 않을 경우, 질문을 재정의합니다.
6. **Web Search**: 재정의된 질문을 기반으로 Web Search를 수행합니다.
7. **Generate**: 검색된 내용을 질문과 함께 LLM에 입력하여 답변을 생성합니다.
8. **그래프화**: 앞서 생성한 기능들을 그래프화 시켜 하나의 RAG 파이프라인으로 구현합니다.

위 내용들을 자유롭게 Customizing 하여 나만의 LangGraph를 구현합니다.

### 2-1) 문서를 다운받고 DB 및 검색기를 생성합니다.

In [ ]:
# 실습을 진행할 OpenAI o1 System Card 논문 다운로드
!wget -O openai-o1-system-card.pdf https://cdn.openai.com/o1-system-card-20241205.pdf

--2024-12-19 13:44:19--  https://cdn.openai.com/o1-system-card-20241205.pdf
Resolving cdn.openai.com (cdn.openai.com)... 13.107.246.40, 2620:1ec:29:1::40
Connecting to cdn.openai.com (cdn.openai.com)|13.107.246.40|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4533906 (4.3M) [application/pdf]
Saving to: ‘openai-o1-system-card.pdf’

openai-o1-system-ca 100%[===================>]   4.32M  --.-KB/s    in 0.08s   

2024-12-19 13:44:19 (51.7 MB/s) - ‘openai-o1-system-card.pdf’ saved [4533906/4533906]



In [ ]:
# Document Parse로 다운로드 된 문서 불러오기
from langchain_upstage import UpstageDocumentParseLoader

layzer = UpstageDocumentParseLoader(
    "openai-o1-system-card.pdf", # 불러올 파일
    output_format='html',  # 결과물 형태 : HTML
    coordinates= False) # 이미지 OCR 좌표계 가지고 오지 않기

# 약 50초 소요
docs = layzer.load()

In [ ]:
# chunk_size와 chunk_overlap에 따른 성능 비교해봤던 것 기억하시죠!?
# 여기서는 다른 값들에 따른 성능을 비교해봅시다!
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=50)

splits = text_splitter.split_documents(docs)

print("Splits:", len(splits))

Splits: 1030


In [ ]:
%%time

from langchain_chroma import Chroma
from langchain_upstage import UpstageEmbeddings

# Chroma 활용하여 vectorstore 만들기
chroma_vectorstore = Chroma.from_documents(
     documents=splits, embedding=UpstageEmbeddings(model="embedding-query")
)

retriever = chroma_vectorstore.as_retriever(
    search_type= 'mmr', # default : similarity(유사도) / mmr 알고리즘
    search_kwargs={"k": 3} # 쿼리와 관련된 chunk를 3개 검색하기 (default : 4)
)

In [ ]:
questions = [
'What distinguishes the o1 model’s reasoning capabilities from previous OpenAI models, and how does "chain-of-thought" improve its performance?',
'How does the o1 model perform in jailbreak evaluations compared to GPT-4o, and what measures have been implemented to resist adversarial prompts?',
'What are the key improvements in the o1 model regarding hallucinations and fairness compared to GPT-4o, and how were these measured?',
'What safety challenges arise from o1’s enhanced reasoning abilities, and how does OpenAI address the risks associated with chain-of-thought reasoning?',
'What were the findings from external red teaming efforts, particularly concerning the o1 model’s susceptibility to manipulation, persuasion, and scheming behaviors?'
]

user_question = questions[0]

### 2-2) **Retrieve**: 질문에 관련된 문서를 검색합니다.


In [ ]:
docs = retriever.get_relevant_documents(user_question)
docs = [doc.page_content for doc in docs]

<ipython-input-28-0eedde740f81>:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(user_question)


### 2-3) **Grade**: 검색된 문서가 적절한 지 판단합니다.


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_upstage import ChatUpstage
from langchain.text_splitter import RecursiveCharacterTextSplitter

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3553: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
# Data model
class GradeDocuments(BaseModel):
    """ 검색된 문서가 질문과 관련이 있는 지 혹은 없는 지 확인합니다.Binary score for relevance check on retrieved documents."""
    binary_score: str = Field(
        description="Documents are relevant to the question, 'yes' or 'no'"
    )


# LLM with function call
llm = ChatUpstage()
structured_llm_grader = llm.with_structured_output(GradeDocuments)

# Prompt
system = """You are a grader assessing relevance of a retrieved document to a user question. \n
    If the document contains keyword(s) or semantic meaning related to the question, grade it as relevant. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."""

user = """<<<Retrieved document>>>
{document}

<<<User question>>>
{question}

<<<Output Format>>>
`Score: <yes or no>`
"""

grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", user),
    ]
)
retrieval_grader = grade_prompt | structured_llm_grader

In [ ]:
# LLM Output Formatting 이슈로 에러가 발생할 수 있습니다. 어떻게 해결할 수 있을 지 생각해봅니다.
for doc in docs:
    score = retrieval_grader.invoke({"question": user_question, "document": doc})
    grade = score.binary_score
    print(grade)

yes
yes
no


### 2-4) **Generate**: 적절할 경우, 질문과 함께 LLM에 입력하여 답변을 생성합니다.


In [ ]:
# yes가 2개 이상일 경우, 질문과 함께 LLM에 입력하여 답변을 획득합니다.
from langchain_core.output_parsers import StrOutputParser

# Prompt
system = """
Answer the question based on context.
"""
user = """
Question: {question}
Context: {context}

<<<Output Format>>>
`Answer: <Answer based on the document.>`
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", user),
    ]
)

# LLM & Chain
llm = ChatUpstage()
rag_chain = prompt | llm | StrOutputParser()

In [ ]:
# Generate
generation = rag_chain.invoke({"context": "\n\n".join(docs), "question": user_question})
generation = generation.split(":")[1].strip() if ":" in generation else generation.strip()
print(generation)

The o1 model's reasoning capabilities are distinguished from previous OpenAI models by being trained with large-scale reinforcement learning to reason using a "chain-of-thought" approach. This improves its performance by providing new avenues for improving safety and strong reasoning in context, although it may reduce performance on some subtasks, such as reimplementing the OpenAI API.


### 2-5) **Re-Write**: 적절하지 않을 경우, 질문을 재정의합니다.

In [ ]:
### Question Re-writer

# Prompt
system = """
You a question re-writer that converts an input question to a better version that is optimized
for web search. Look at the input and try to reason about the underlying semantic intent / meaning."""

user = """
Here is the initial question: {question}
Formulate an improved question.

<<<Output Format>>>
`new question: <new question that is optimized for the web search>`
"""

re_write_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", user),
    ]
)
llm = ChatUpstage()
question_rewriter = re_write_prompt | llm | StrOutputParser()

In [ ]:
user_question

'What distinguishes the o1 model’s reasoning capabilities from previous OpenAI models, and how does "chain-of-thought" improve its performance?'

In [ ]:
new_question = question_rewriter.invoke({"question": user_question})
new_question = new_question.split(":")[1].strip() if ":" in new_question else new_question.strip()

# web_search에서 '"'이 붙어있을 경우, 검색이 안되는 이슈가 존재합니다.
if new_question.startswith('"'):
    new_question = new_question[1:]
if new_question.endswith('"'):
    new_question = new_question[:-1]

print(new_question)

What makes o1 model's reasoning unique compared to previous OpenAI models and how does the "chain-of-thought" approach enhance its performance?`


### 2-6) **Web Search**: 재정의된 질문을 기반으로 Web Search를 수행합니다.

In [ ]:
from serpapi import GoogleSearch
from langchain.document_loaders import UnstructuredURLLoader

params = {
    "engine": "google",
    "q": questions[0],
    "num": "2"
}
search = GoogleSearch(params)
search_result = search.get_dict()

# URL은 여러개 사용할 수 있다.
urls = [result['link'] for result in search_result['organic_results']]

loader = UnstructuredURLLoader(urls=urls)
data = loader.load()

new_docs = [d.page_content for d in data]

In [ ]:
new_docs

['Open in app\n\nSign in\n\nWrite\n\nSign in\n\nHow OpenAI’s O1 Series Stands Out Redefining AI Reasoning\n\nExploring whether OpenAI’s O1 models represent a true breakthrough in AI reasoning or just incremental progress\n\nResearch Graph\n\nFollow\n\n19 min read\n\nSep 20, 2024\n\n--\n\nAuthor\n\nZijian Yang (ORCID: 0009–0006–8301–7634)\n\nIntroduction\n\nOpenAI’s recently released O1 series models signify a significant step forward in the field of artificial intelligence, particularly in complex reasoning. The O1 model series not only handles more intricate problems but also introduces human-like thought processes, elevating the model’s reasoning capabilities to new heights. In this article, we will delve into the core features of the O1 series models.\n\nThe O1 series models from OpenAI include two main versions, each designed to meet different needs:\n\nO1-preview: This is the preview version of O1, specifically designed for complex reasoning tasks. It excels in handling high-diffi

### 2-7) **Generate**: 검색된 내용을 질문과 함께 LLM에 입력하여 답변을 생성합니다.

In [ ]:
# Run
generation = rag_chain.invoke({"context": "\n\n".join(new_docs), "question": user_question})
generation = generation.split(":")[1].strip() if ":" in generation else generation.strip()
print(generation)

The o1 model’s reasoning capabilities are distinguished from previous OpenAI models by its training using a new optimization algorithm and a new training dataset specifically tailored for it. The model is also trained to solve problems on its own using a technique known as reinforcement learning and uses a "chain of thought" to process queries, similar to how humans process problems step-by-step. This improves its performance by making it more accurate and better at solving complex problems, such as coding and math, while also explaining its reasoning.`


### 2-8) **그래프화**: 앞서 생성한 기능들을 그래프화 시켜 하나의 RAG 파이프라인으로 구현합니다.

앞서 생성한 `retriever`, `retrieval_grader`, `rag_chain`, `question_rewriter`을 활용합니다.

In [ ]:
def retrieve(state):
    """ (2) Retrieve: 질문에 관련된 문서를 검색합니다. """
    print("<<<RETRIEVE>>>")

    user_question = state["user_question"]
    documents = retriever.get_relevant_documents(user_question)
    documents = [doc.page_content for doc in documents]

    return {"documents": documents, "user_question": user_question}


def grade_documents(state):
    """ (3) Grade: 검색된 문서가 적절한 지 판단합니다. """
    print("\n<<<CHECK DOCUMENT RELEVANCE TO QUESTION>>>")

    user_question = state["user_question"]
    docs = state["documents"]

    # 검색된 각 문서마다 통과 여부를 확인합니다.
    filtered_docs = []
    need_web_search = "No"
    for doc in docs:
        score = retrieval_grader.invoke(
            {"question": user_question, "document": doc}
        )
        grade = score.binary_score

        if grade == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(doc)
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            need_web_search = "Yes"

    return {"documents": filtered_docs, "user_question": user_question, "web_search": need_web_search}


def decide_to_generate(state):
    """ 검색을 종료하고 생성할 지 혹은 웹 검색을 수행할 지 결정합니다. """
    print("\n<<<ASSESS GRADED DOCUMENTS>>>")

    need_web_search = state["web_search"]

    # 검색이 필요할 경우 (1개라도 만족되지 않은 문서가 검색되었을 경우)
    if need_web_search == "Yes":
        print("---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, TRANSFORM QUERY---")
        return "query_rewrite"

    # 검색된 모든 문서가 만족되었을 경우
    else:
        print("---DECISION: GENERATE---")
        return "generate"


def generate(state):
    """ (4) Generate: 적절할 경우, 질문과 함께 LLM에 입력하여 답변을 생성합니다. """
    print("\n<<<GENERATE>>>")

    user_question = state["user_question"]
    documents = state["documents"]

    # RAG generation
    generation = rag_chain.invoke({"context": "\n\n".join(documents), "question": user_question})
    generation = generation.split(":")[1].strip() if ":" in generation else generation.strip()

    return {"documents": documents, "user_question": user_question, "generation": generation}


def query_rewrite(state):
    """ (5) Re-Write: 적절하지 않을 경우, 질문을 재정의합니다. """
    print("\n<<<TRANSFORM QUERY>>>")

    user_question = state["user_question"]
    documents = state["documents"]

    # Re-write question
    web_question = question_rewriter.invoke({"question": user_question})
    web_question = web_question.split(":")[1].strip() if ":" in web_question else web_question.strip()

    # web_search에서 '"'이 붙어있을 경우, 검색이 안되는 이슈가 존재합니다.
    if web_question.startswith('"'):
        web_question = web_question[1:]
    if web_question.endswith('"'):
        web_question = web_question[:-1]

    return {"documents": documents, "user_question": web_question}

def search_docs(state):
    """ (6) Web Search: 재정의된 질문을 기반으로 Web Search를 수행합니다. """

    print("\n<<<SEARCH CONVERSATIONS>>>")
    user_question = state["user_question"]
    documents = state['documents']

    params = {
        "engine": "google",
        "q": user_question,
        "num": "2"
    }
    search = GoogleSearch(params)
    search_result = search.get_dict()

    # URL은 여러개 사용할 수 있다.
    urls = [result['link'] for result in search_result['organic_results']]

    loader = UnstructuredURLLoader(urls=urls)
    data = loader.load()

    new_documents = [d.page_content for d in data]

    return {"documents": new_documents, "user_question": user_question}


In [ ]:
from typing import List
from typing_extensions import TypedDict

class GraphState(TypedDict):
    """ 그래프의 state로 들어갈 항목 정의 """
    user_question: str
    documents: List[str]
    web_search: str
    generation: str

In [ ]:
from langgraph.graph import END, StateGraph, START

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("retrieve", retrieve)  # retrieve
workflow.add_node("grade_documents", grade_documents)  # grade documents
workflow.add_node("generate", generate)  # generatae
workflow.add_node("search_docs", search_docs)  # search_conversation
workflow.add_node("query_rewrite", query_rewrite)  # query_rewrite

# Build graph
workflow.add_edge(START, "retrieve")
workflow.add_edge("retrieve", "grade_documents")
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "query_rewrite": "query_rewrite",
        "generate": "generate",
    },
)
workflow.add_edge("query_rewrite", "search_docs")
workflow.add_edge("search_docs", "generate")
workflow.add_edge("generate", END)

# Compile
app = workflow.compile()

In [ ]:
from pprint import pprint

# Run
# NOTE: Grade에서 포매팅 이슈로 에러가 발생할 수 있습니다. (재실행)
user_question = questions[1]
inputs = {"user_question": user_question}
for output in app.stream(inputs):
    for key, value in output.items():
        pprint(f"Node '{key}'")
    pprint("\n---\n")

# Final generation
pprint(value["generation"])

<<<RETRIEVE>>>
"Node 'retrieve'"
'\n---\n'

<<<CHECK DOCUMENT RELEVANCE TO QUESTION>>>
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---

<<<ASSESS GRADED DOCUMENTS>>>
---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, TRANSFORM QUERY---
"Node 'grade_documents'"
'\n---\n'

<<<TRANSFORM QUERY>>>
"Node 'query_rewrite'"
'\n---\n'

<<<SEARCH CONVERSATIONS>>>
"Node 'search_docs'"
'\n---\n'

<<<GENERATE>>>
"Node 'generate'"
'\n---\n'
('The O1 model performs better in jailbreak evaluations compared to GPT-4o, '
 'and measures have been implemented to resist adversarial prompts by using '
 'reasoning tokens and a chain of thought process.`')


In [ ]:
print("답변:\n", value["generation"])

답변:
 The O1 model performs better in jailbreak evaluations compared to GPT-4o, and measures have been implemented to resist adversarial prompts by using reasoning tokens and a chain of thought process.`


# 🚀 실습 마무리

이번 실습을 통해 실시간 검색 엔진을 활용한 최신 정보를 수진하는 기능과 LangGraph를 통한 RAG 파이프라인 고도화를 배웠습니다.

LangGraph를 활용하는 방법은 위의 방법 말고도 다양한 방식들이 있을 수 있습니다. 본인만의 Custom된 Graph를 활용하여 본인만의 RAG 파이프라인을 만들어봅시다.

이러한 기술들을 십분 활용하여 각자 설계한 서비스가 목표에 맞도록 구현될 수 있기를 바랍니다.